In [1]:
import torch
import torch.nn as nn
from abc import ABC, abstractmethod

class ClassificationModel(ABC, nn.Module):
    def __init__(self):
        super(ClassificationModel, self).__init__()

    @abstractmethod
    def forward(self, x):
        """
        Forward pass of the model.
        """
        pass

    @abstractmethod
    def train_model(self, dataloader, criterion, optimizer, num_epochs):
        """
        Method to train the model.
        """
        pass

    @abstractmethod
    def evaluate_model(self, dataloader, criterion):
        """
        Method to evaluate the model.
        """
        pass

    @abstractmethod
    def predict(self, x):
        """
        Method to make predictions.
        """
        pass

    def save_model(self, path):
        """
        Save the model to a file.
        """
        torch.save(self.state_dict(), path)

    def load_model(self, path):
        """
        Load the model from a file.
        """
        self.load_state_dict(torch.load(path))


In [3]:
class MyCustomModel(ClassificationModel):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MyCustomModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

    def train_model(self, dataloader, criterion, optimizer, num_epochs):
        for epoch in range(num_epochs):
            for inputs, labels in dataloader:
                optimizer.zero_grad()
                outputs = self(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    
    def evaluate_model(self, dataloader, criterion):
        total_loss = 0.0
        total_correct = 0
        total_samples = 0
        with torch.no_grad():
            for inputs, labels in dataloader:
                outputs = self(inputs)
                loss = criterion(outputs, labels)
                total_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total_samples += labels.size(0)
                total_correct += (predicted == labels).sum().item()
        accuracy = 100.0 * total_correct / total_samples
        print(f'Loss: {total_loss:.4f}, Accuracy: {accuracy:.2f}%')
    
    def predict(self, x):
        with torch.no_grad():
            outputs = self(x)
            _, predicted = torch.max(outputs, 1)
        return predicted

# Now you can instantiate the custom model class
model = MyCustomModel(input_size=784, hidden_size=128, num_classes=10)


In [4]:
model

MyCustomModel(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)